<a href="https://colab.research.google.com/github/SomeiLam/langchain-example/blob/main/LangChain_chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install langchain_community
!pip install -U langchain-openai

In [6]:
import os
import openai
from google.colab import userdata
api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
!pip install pandas

In [8]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [9]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [56]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

LangChain's LLMChain has been deprecate as of version 0.1.17 in favor of the "Runnable" API. Instead of import LLMChain from langchain.chains, we now compose prompts and models as runnables, chaining them with the pipe operator (|) or via `RunnableSequence`. We can migrate it as:

In [11]:
from langchain.schema.runnable import RunnableSequence, RunnableLambda

### What is RunnableSequence?
A `RunnableSequence` is the core composition primitive in LangChain’s new Runnable API. It implements the `Runnable` interface and simply runs a series of smaller “runnables” one after the other, piping the output of each into the next. Under the hood:

* It implements the standard methods—`.invoke()` / `.ainvoke()`, `.batch()` / `.abatch()`, and streaming variants—so you get sync, async, batch, and streaming support “for free.”

* You can construct one explicitly:
```python
from langchain.schema.runnable import RunnableSequence
chain = RunnableSequence(first=prompt, middle=[], last=llm)
```
* Most commonly, you build a RunnableSequence with the pipe operator (|), which is syntactic sugar for the same thing:
```python
chain = prompt | llm
```
Under the hood it is exactly equivalent to:
```python
from langchain.schema.runnable import RunnableSequence
chain = RunnableSequence(prompt, llm)
```
Here’s what happens step‑by‑step:

1. prompt (a ChatPromptTemplate) is a Runnable that, given {"product": "X"}, produces a list of chat messages (the filled‑in prompt).

2. llm (a ChatOpenAI) is a Runnable that, given a list of chat messages, sends them to the model and returns a chat response.

3. Chaining with | wires the output of the left Runnable into the input of the right Runnable, producing a new RunnableSequence.

You get the same behavior—and identical `.invoke()` interface—but with far more readable, “pipe‑style” syntax. You can chain as many steps as you like:
```python
# e.g. format → generate → parse → store
chain = prompt | llm | MyParserRunnable() | MyStorageRunnable()
```

This wires the prompt’s output into the LLM’s input, creating a single composite runnable


---



Because it’s itself a Runnable, you can treat the entire chain just like any other primitive:
```python
# Synchronous
response = (prompt | llm).invoke({"product": "eco-friendly sneakers"})
print(response.content)

# Asynchronous
response = await (prompt | llm).ainvoke({"product": "eco-friendly sneakers"})
```

Behind the scenes `RunnableSequence` handles validation, batching, and streaming. It’s the building block that replaces `LLMChain`, giving you much more flexibility and composability.


Define the LLM and prompt:

In [57]:
# ChatOpenAI is a Runnable for OpenAI’s chat models
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9)

# ChatPromptTemplate.from_template yields a Runnable prompt template
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

Chain them together:

In [13]:
# Option A: using the pipe operator
chain = prompt | llm

# Option B: explicitly building a RunnableSequence
chain = RunnableSequence(prompt, llm)

Running the chain:

In [14]:
# get the full invoke output
output = chain.invoke({"product": "Queen Size Sheet Set"})
print(output)  # e.g. {'text': 'Greensole', ...}

content='Royal Comfort Sheets' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 23, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-6ca68340-d7cb-468a-ae10-df27501e80a0-0'


## SimpleSequentialChain

In [15]:
from langchain.chains import SimpleSequentialChain

In [16]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = RunnableSequence(
    first_prompt,
    llm,
    RunnableLambda(
      lambda msg: {"company_name": msg.content}
    ))

# Option 2: use the pipe operator (syntactic sugar)
chain_one = first_prompt | llm | RunnableLambda(
    lambda msg: {"company_name": msg.content})

In [17]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = RunnableSequence(second_prompt, llm)

# Option 2: use the pipe operator (syntactic sugar)
chainchain_two_one = second_prompt | llm

In [18]:
product = "Queen Size Sheet Set"
#extract_company = RunnableLambda(lambda msg: {"company_name": msg.content})
overall_simple_chain = RunnableSequence(
    chain_one,
    # extract_company,
    chain_two,
)

In [19]:
result = overall_simple_chain.invoke({"product": "Queen Size Sheet Set"})
# if the prompt template only defines a single input variable, we can pass that
# value directly to .invoke(...)
# overall_simple_chain.invoke("Queen Size Sheet Set")
print(result.content)

"Royal Rest Bedding Co. offers luxurious, comfortable mattresses and bedding products fit for royalty, guaranteeing a good night's sleep."


## SequentialChain

In [20]:
from langchain.chains import SequentialChain

In [21]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = RunnableSequence(first_prompt, llm)

In [22]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = RunnableSequence(second_prompt, llm)

In [23]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = RunnableSequence(third_prompt, llm)

In [24]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = RunnableSequence(fourth_prompt, llm)


The lambda func passing into `RunnableLambda` is a one‑step “context merger” that:

1. Takes the current context dict as its sole argument, which we’re calling ctx.

2. Unpacks every key/value pair from that dict into a brand‑new dict via **ctx.

3. Adds one more entry under the key "English_Review", whose value is computed by running your translation chain on the original review text (ctx["Review"]) and pulling out its .content.

```python
lambda ctx: {
  **ctx, # copy in everything already in ctx
  "English_Review": chain_one.invoke(ctx["Review"]).content
}
```
* `ctx["Review"]` fetches the raw review string.

* `chain_one.invoke(...)` runs your first prompt→LLM pipeline and returns a BaseMessage.

* `.content` extracts the generated string from that message.

* The resulting dict now has every key that was in `ctx` plus `"English_Review": "<translated text>"`.


In [25]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message

# A) seed the context with the raw Review string
seed = RunnableLambda(lambda review: {"Review": review})

# B) run translation, append "English_Review" but keep full context
chain_one_ctx = RunnableLambda(
    lambda ctx: {
        **ctx,
        "English_Review": chain_one.invoke(ctx["Review"]).content
    }
)

# C) run summarization, append "summary"
chain_two_ctx = RunnableLambda(
    lambda ctx: {
        **ctx,
        "summary": chain_two.invoke(ctx["English_Review"]).content
    }
)

# D) inject your desired language for the follow‑up
#    (you could also parameterize this or read from ctx, but here it’s hard‑coded)
chain_three_ctx = RunnableLambda(
    lambda ctx: {
        **ctx,
        "language": chain_three.invoke(ctx["Review"]).content
    }
)

# E) run follow‑up, append "followup_message"
chain_four_ctx = RunnableLambda(
    lambda ctx: {
        **ctx,
        "followup_message": chain_four.invoke({
            "summary":  ctx["summary"],
            "language": ctx["language"]
        }).content
    }
)

# Finally stitch them all together:
overall_chain = RunnableSequence(
    seed,
    chain_one_ctx,
    chain_two_ctx,
    chain_three_ctx,
    chain_four_ctx,
)

In [26]:
review = df.Review[5]
review
overall_chain.invoke(review)

{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste poor. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': 'The reviewer is dissatisfied with the poor taste and quality of the product, suspecting that it may be an old batch or counterfeit.',
 'language': 'French',
 'followup_message': "Cher client,\n\nNous sommes profondément désolés d'apprendre que vous n'êtes pas satisfait de votre récente expérience avec notre produit. Nous prenons vos préoccupations très au sérieux et nous voulons vous assurer que la qualité de nos produits est notre priorité absolue.\n\nNous comprenons vos inquiétudes concernant la possibilité d'un lot périmé ou contrefait et nous prenons des mesures immédiates pour enquêter sur la situation. Si vous le souhaitez, n

## Router Chain

In [27]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [28]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [29]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableSequence

In [30]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9)

In [40]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = RunnableSequence(prompt, llm)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [41]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = RunnableSequence(default_prompt, llm)

In [34]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [44]:
map_for_router = RunnableLambda(lambda d: {
    "key":   d["destination"],
    "input": d["next_inputs"],
})
map_for_router

RunnableLambda(lambda d: {'key': d['destination'], 'input': d['next_inputs']})

In [50]:
from langchain_core.runnables.router import RouterRunnable

router = RouterRunnable(
    runnables={
        **destination_chains,       # your named branches
        "DEFAULT": default_chain,   # the fallback branch
    }
)

overall_chain = RunnableSequence(
    router_chain,    # picks {"destination","next_inputs"}
    RunnableLambda(lambda d: {"key": d["destination"], "input": d["next_inputs"]}),
    router,          # runs only runnables[key], falling back to "DEFAULT"
)
result = overall_chain.invoke({"input": "Tell me about the Pythagorean theorem."})
result.content

"The Pythagorean theorem is a fundamental principle in geometry that states that in a right-angled triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. In equation form, it is written as: \n\na^2 + b^2 = c^2\n\nWhere 'a' and 'b' are the lengths of the two shorter sides of the triangle, and 'c' is the length of the hypotenuse. This theorem is named after the ancient Greek mathematician Pythagoras, although it is believed that it was known to the Babylonians even earlier.\n\nThe Pythagorean theorem has many practical applications in various fields such as physics, engineering, and architecture. It is used to calculate distances, solve equations involving right triangles, and even in the development of computer graphics algorithms. It is one of the most well-known and widely used mathematical principles in the world."

In [49]:
def wrap_branch(chain, key_name):
    # Takes {"input":...} → runs chain → returns {"destination":key_name, "response": <text>}
    return RunnableSequence(
        chain,  # expects a single string input or {"input": ...}
        RunnableLambda(lambda msg, *, key=key_name: {
            "destination": key,
            "response": msg.content
        })
    )
    # wrap all your branches (and default) under the same dict keys
wrapped = {
    **{k: wrap_branch(c, k) for k, c in destination_chains.items()},
    "DEFAULT": wrap_branch(default_chain, "DEFAULT"),
}

router = RouterRunnable(runnables=wrapped)

overall = RunnableSequence(
    # step A: pick destination & inputs
    router_chain,
    # step B: reshape into {"key","input"} for RouterRunnable
    RunnableLambda(lambda d: {"key": d["destination"], "input": d["next_inputs"]}),
    # step C: run only the chosen wrapped branch
    router,
)

out = overall.invoke("How do I train a neural network?")
# out == {"destination": "computer science", "response": "You train by…"}
print("Chose branch:", out["destination"])
print("Got response:", out["response"])

Chose branch: computer science
Got response: Training a neural network involves several steps:

1. Data collection: Gather a dataset that includes input data and corresponding output data.

2. Data preprocessing: Clean and preprocess the data to ensure it is in a suitable format for training the neural network.

3. Choose a neural network architecture: Decide on the number of layers, type of activation functions, and other parameters for the neural network.

4. Initialize the weights and biases: Randomly initialize the weights and biases for the neural network.

5. Forward propagation: Pass input data through the neural network to compute the output.

6. Calculate the loss: Compare the output of the neural network with the actual output and calculate the loss using a loss function.

7. Backward propagation (backpropagation): Adjust the weights and biases of the neural network to minimize the loss using gradient descent or other optimization algorithms.

8. Update the weights and biases

## RouterRunnable inside a RunnableSequence

When we need our application to choose among multiple “next steps”—for example, different LLM prompts or models—based on the user’s input, you can use LangChain’s `RouterRunnable` inside a `RunnableSequence` to build a single, composable pipeline:

### 1. Core concepts
* **RunnableSequence:**
A composable pipeline of one or more “runnables” (prompts, models, lambdas, routers). It takes the output of each step and feeds it into the next, supporting sync (`.invoke()`), async (`.ainvoke()`), batch, and streaming APIs uniformly.

* **RouterRunnable:**
A special runnable that, given a dict with `{"key": <branch‑name>, "input": <user‑text>}`, looks up `self.runnables[key]` in its internal mapping and runs only that one branch’s runnable on the given input. If the key isn’t found, it falls back to the "DEFAULT" branch.

* **Mapping step:**
Before calling a `RouterRunnable`, we must reshape our router’s output (often from an `LLMRouterChain`) into the exact `{ "key": ..., "input": ... }` shape the router expects. A tiny `RunnableLambda` does the trick.

### 2. Step-by-step breakdown
1. **Route** -
We run an LLMRouterChain (or any model) with a “multi‑prompt router” template. Its output is a dict like:
```python
{
  "destination": "math",
  "next_inputs": "Compute the derivative of x^2"
}
```

2. **Reshape** -
Convert that into the router’s format:
```python
{"key": destination, "input": next_inputs}
```
via:
```python
map_for_router = RunnableLambda(lambda d: {
    "key":   d["destination"],
    "input": d["next_inputs"],
})
```

3. **Dispatch** -
A single `RouterRunnable` is constructed with a mapping of all our branches (plus a `"DEFAULT"` fallback):
```python
router = RouterRunnable(runnables={
  "physics":     physics_chain,
  "math":        math_chain,
  "history":     history_chain,
  "computer science": cs_chain,
  "DEFAULT":     default_chain
})
```

4. **Compose** -
Put it all inside one RunnableSequence so we can call it in one shot:
```python
overall = RunnableSequence(
  router_chain,    # Step A: pick branch & next_inputs
  map_for_router,  # Step B: reshape to {"key","input"}
  router,          # Step C: run only the chosen branch
)
```

5. **Invoke** -
```python
# Synchronous
result = overall.invoke({"input": "How do I solve integrals?"})
print(result.content)  # e.g. the math_chain’s answer
# Asynchronous
# result = await overall.ainvoke("What caused the fall of Rome?")
```

### 3. Full example code
```python
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableSequence, RunnableLambda
from langchain_core.runnables.router import RouterRunnable
from langchain.chains.router import LLMRouterChain

# 1) Define your base LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# 2) Build your router prompt & chain
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input...
<< CANDIDATE PROMPTS >>
physics: for physics questions
math:     for math questions
history:  for history questions
computer science: for CS questions
<< INPUT >>
{input}

<< OUTPUT (JSON)>>"""
router_prompt = ChatPromptTemplate.from_template(MULTI_PROMPT_ROUTER_TEMPLATE)
router_chain = LLMRouterChain.from_llm(llm=llm, router_prompt=router_prompt)

# 3) Map the router’s output into {"key","input"}
map_for_router = RunnableLambda(lambda d: {
    "key":   d["destination"],
    "input": d["next_inputs"],
})

# 4) Define each branch as a prompt|llm (they’re Runnables too!)
destination_chains = {
    "physics": (ChatPromptTemplate.from_template("Physics answer: {input}") | llm),
    "math":    (ChatPromptTemplate.from_template("Math answer: {input}")    | llm),
    "history": (ChatPromptTemplate.from_template("History answer: {input}") | llm),
    "computer science": (
       ChatPromptTemplate.from_template("CS answer: {input}") | llm
    ),
}

# 5) Add a DEFAULT fallback
default_chain = ChatPromptTemplate.from_template("General: {input}") | llm

# 6) Create the RouterRunnable with all branches
router = RouterRunnable(runnables={**destination_chains, "DEFAULT": default_chain})

# 7) Stitch into one sequence
overall_chain = RunnableSequence(
    router_chain,    # A: decide branch
    map_for_router,  # B: reshape output
    router,          # C: dispatch chosen branch
)

# 8) Run it!
query = "Can you explain Newton's laws?"
response = overall_chain.invoke({"input": query})
print("Branch answer:", response.content)

```

### Key points
* **Single entrypoint.** Our application only ever calls `overall_chain.invoke()` (or `.ainvoke()`), regardless of how many branches you have.

* **Easy to extend.** To add a new topic, just add a new key→runnable in the `runnables` dict.

* **Fully composable.** You can pipe more steps before or after—logging, caching, error handling, whatever—simply by adding more runnables to the `RunnableSequence`.